In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [4]:
from pyspark.sql.functions import to_date

# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
|R3NORADVJO6IE6|          5|            0|          0|   N|                Y|
|R299F4SO98S5OO|          5|            1|          1|   N|                Y|
|R2DA9DOT03UW6I|          5|            2|          3|   N|                Y|
|R2OGCH681EQHU6|          5|            0|          0|   N|                Y|
| R2JMKIC16MHD7|          5|            2|          2|   N|                Y|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|                Y|
|R1N8XWFDK4QACP|          5|            0|          0|   N|                Y|
|R19VFLX6MRU0CX|          5|            0|          0|   N|                Y|
| RLTD8YDGTP9PT|          1|            0|          0|   N|     

In [7]:
# Filter Vine table data to keep only rows where total votes >= 20

vine_df_filtered = vine_df.filter(vine_df["total_votes"] >= 20)
vine_df_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1T4FVSABO0IDP|          5|            8|         31|   N|                Y|
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R3HAK0MUN9F5IT|          1|            6|         28|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|     

In [11]:
# Filter resulting DF to keep only reviews with a positive ratio of helpful votes

vine_df_filtered_positive = vine_df_filtered.filter(vine_df_filtered["helpful_votes"] / vine_df_filtered["total_votes"] >= 0.5)
vine_df_filtered_positive.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R13YNG792BCR77|          5|           19|         20|   N|                Y|
|R1RYG8MVOPK4Q2|          5|           18|         20|   N|     

In [12]:
vine_df_paid = vine_df_filtered_positive.filter(vine_df_filtered_positive["vine"] == "Y")
vine_df_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REKN4PLM5174P|          4|          219|        225|   Y|                N|
| RQOG8IIJX7CKB|          4|           21|         27|   Y|                N|
|R33EJ9PSZR13CM|          4|           26|         30|   Y|                N|
| RD2BFV896F321|          5|           16|         20|   Y|                N|
|R37GSXLA11XF8N|          5|           30|         34|   Y|                N|
| RQBQYBM9VMSH4|          5|           19|         20|   Y|                N|
|R2YE4F9ESKFWEK|          3|           20|         23|   Y|                N|
|R1E4L7HHB8TBP9|          4|           80|         92|   Y|                N|
|R2C2WXDXMZL005|          2|           20|         25|   Y|                N|
|R1PPY4U0K03KTE|          5|           31|         38|   Y|     

In [13]:
vine_df_unpaid = vine_df_filtered_positive.filter(vine_df_filtered_positive["vine"] == "N")
vine_df_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R13YNG792BCR77|          5|           19|         20|   N|                Y|
|R1RYG8MVOPK4Q2|          5|           18|         20|   N|     

In [19]:
# Analysis - paid reviews
paid_total_reviews = vine_df_paid.count()
paid_total_fivestar_reviews = vine_df_paid.filter(vine_df_paid["star_rating"] == 5).count()
paid_pct_fivestar_reviews = round((paid_total_fivestar_reviews / paid_total_reviews * 100), 2)
print(f"Paid Reviews Analysis -- Total Reviews: {paid_total_reviews}, Total 5-Star Reviews: {paid_total_fivestar_reviews}, Percentage of 5-Star Reviews: {paid_pct_fivestar_reviews}%")

Paid Reviews Analysis -- Total Reviews: 82, Total 5-Star Reviews: 33, Percentage of 5-Star Reviews: 40.24%


In [21]:
# Analysis - unpaid reviews
unpaid_total_reviews = vine_df_unpaid.count()
unpaid_total_fivestar_reviews = vine_df_unpaid.filter(vine_df_unpaid["star_rating"] == 5).count()
unpaid_pct_fivestar_reviews = round((unpaid_total_fivestar_reviews / unpaid_total_reviews * 100), 2)
print(f"Unpaid Reviews Analysis -- Total Reviews: {unpaid_total_reviews}, Total 5-Star Reviews: {unpaid_total_fivestar_reviews}, Percentage of 5-Star Reviews: {unpaid_pct_fivestar_reviews}%")

Unpaid Reviews Analysis -- Total Reviews: 24742, Total 5-Star Reviews: 12807, Percentage of 5-Star Reviews: 51.76%


In [26]:
# Create summary DF
summary_df = spark.createDataFrame([
                                   ("Paid",paid_total_reviews,paid_total_fivestar_reviews,str(paid_pct_fivestar_reviews)+"%"),
                                   ("Unpaid",unpaid_total_reviews,unpaid_total_fivestar_reviews,str(unpaid_pct_fivestar_reviews)+"%"),
], ["Review Class","Total Reviews", "Total 5-Star Reviews", "Percentage of 5-Star Reviews"]).toPandas()
summary_df

,Review Class,Total Reviews,Total 5-Star Reviews,Percentage of 5-Star Reviews
0,Paid,82,33,40.24%
1,Unpaid,24742,12807,51.76%
